In [1]:
import neo4j
import time
from neo4j import GraphDatabase
uri = "neo4j+ssc://347bdfd6.production-orch-0002.neo4j.io:7687"
username = "neo4j"
password = "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc"
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

def nearest_node(location):
  lat = location[0]
  longit = location[1]
  query = "MATCH (n:LOCATION) RETURN n ORDER BY (n.x - "+str(lat)+") * (n.x - "+str(lat)+")" + " + (n.y - " + str(longit)+")*(n.y - "+str(longit)+")  ASC LIMIT 1"
  closest = session.run(query)
  c_dict = dict()
  for c in closest:
    c_dict = dict(c["n"])
  print("Nearest Node: ", (c_dict["x"], c_dict["y"]))
  return (c_dict["x"], c_dict["y"])

def path_finder_simple(slong: str, slat: str, elong: str, elat: str):
  start = [float(slong),float(slat)]
  end = [float(elong), float(elat)]
  start = nearest_node(start)
  end = nearest_node(end)
  slong = str(start[0])
  slat = str(start[1])
  elong = str(end[0])
  elat = str(end[1])
  max_nodes=15
  while True:
    seconds = time.time()
    print(max_nodes)
    query = "MATCH paths = (a:LOCATION {x:"+str(slong)+"})-[:ROAD*1.."+str(max_nodes)+"]-(b:LOCATION {x:"+str(elong)+"}) WITH paths, relationships(paths) AS rels UNWIND rels AS rel WITH [metro IN nodes(paths) | metro.x] AS long, [metro IN nodes(paths) | metro.y] AS lat, [metro IN nodes(paths) | metro.INTERSECTION] AS intersection, sum(rel.safety_score) AS travelTime RETURN long, lat, intersection ORDER BY travelTime Limit 1"
    results = (session.run(query))  
    path = list()
    final_path = list()
    
    for result in results:
      longs = result["long"]
      lats = result["lat"]
      intersections = result["intersection"]
      print('longs',longs)
      print('lats',lats)
      print('intersections',intersections)
      for long in longs:
        path.append([long])
      for i, lat in enumerate(lats):
        path[i].append(lat)
      for i, intersection in enumerate(intersections):
        if intersection == True:
          final_path.append(path[i])
    delta_time = time.time()-seconds
    print(delta_time)
    print(path)
    if delta_time>2 and path!=[]:
      print('done')
      return jsonify(path)
    if final_path==[]:
      max_nodes += 3
    else:
      max_nodes+=2
    print("\n")



In [2]:
print(path_finder_simple(-122.03687430704345, 36.97406684453328, -122.03805228230542, 36.97415383633685))

Nearest Node:  (-122.037200309277, 36.974015258702295)
Nearest Node:  (-122.03805012501235, 36.9739597045977)
15
longs [-122.037200309277, -122.03727386462762, -122.03756422030929, -122.03771707500897, -122.03805012501235]
lats [36.974015258702295, 36.973941729022194, 36.97365147912628, 36.973749857308725, 36.9739597045977]
intersections [None, None, None, None, None]
1.3802337646484375
[[-122.037200309277, 36.974015258702295], [-122.03727386462762, 36.973941729022194], [-122.03756422030929, 36.97365147912628], [-122.03771707500897, 36.973749857308725], [-122.03805012501235, 36.9739597045977]]


18
longs [-122.037200309277, -122.03727386462762, -122.03756422030929, -122.03771707500897, -122.03805012501235]
lats [36.974015258702295, 36.973941729022194, 36.97365147912628, 36.973749857308725, 36.9739597045977]
intersections [None, None, None, None, None]
14.559904336929321
[[-122.037200309277, 36.974015258702295], [-122.03727386462762, 36.973941729022194], [-122.03756422030929, 36.9736514

NameError: name 'jsonify' is not defined

In [1]:
import neo4j
from neo4j import GraphDatabase
from flask import Flask
from flask import jsonify
from flask_cors import CORS
import time
app = Flask(__name__)
CORS(app)

uri = "neo4j+ssc://347bdfd6.production-orch-0002.neo4j.io:7687"
username = "neo4j"
password = "6uNa34OrH1v3_T08NF-bX7DMaqc55YE--ZUQCHI5zCc"
driver = GraphDatabase.driver(uri, auth=(username, password))
session = driver.session()

def nearest_node(location):
  lat = location[0]
  longit = location[1]
  query = "MATCH (n:LOCATION) RETURN n ORDER BY (n.x - "+str(lat)+") * (n.x - "+str(lat)+")" + " + (n.y - " + str(longit)+")*(n.y - "+str(longit)+")  ASC LIMIT 1"
  closest = session.run(query)
  c_dict = dict()
  for c in closest:
    c_dict = dict(c[0])
  print("Nearest Node: ", (c_dict["x"], c_dict["y"]))
  return (c_dict["x"], c_dict["y"])

def nearest_intersection(location):
  lat = location[0]
  longit = location[1]
  query = "MATCH (n:LOCATION) WHERE n.INTERSECTION = TRUE RETURN n ORDER BY (n.x - "+str(lat)+") * (n.x - "+str(lat)+")" + " + (n.y - " + str(longit)+")*(n.y - "+str(longit)+")  ASC LIMIT 1"
  closest = session.run(query)
  c_dict = dict()
  for c in closest:
    c_dict = dict(c[0])
  return (c_dict["x"], c_dict["y"])

def route(closestart, closeend):
  print("CLOSESTART: ", closestart, ", CLOSEEND: ", closeend)
  prev = dict()
  dist = dict()
  unex_dist = dict()
  unex = set()
  for v in session.run("MATCH (n:LOCATION) RETURN n"):
      try:
        vert = (dict(v["n"])["x"], dict(v["n"])["y"])
      except:
        print(dict(v["n"]))
      dist[vert] = float("INF")
      prev[vert] = None
  print("Completed initialization in Djikstra's.")
  #print("unex: ", unex)
  dist[closestart] = 0
  unex_dist = dict()
  curr_vert = closestart
  unex.add(curr_vert)
  unex_dist[closestart] = 0
  while closeend != curr_vert:
    curr_vert = min(unex_dist, key = unex_dist.get)
    unex.remove(curr_vert)
    unex_dist.pop(curr_vert)
    for w in session.run("MATCH (n:LOCATION)-[r:ROAD]->(m:LOCATION) WHERE n.x = " + str(curr_vert[0]) + " AND n.y = " + str(curr_vert[1]) + " RETURN m, r"):
      wv = (dict(w['m'])["x"], dict(w['m'])["y"])
      if dist[curr_vert] + dict(w["r"])["length"] < dist[wv]:
          dist[wv] = dist[curr_vert] + dict(w["r"])["safety_score"]
          prev[wv] = curr_vert
      unex.add(wv)
      unex_dist[wv] = dist[wv]
  x = closeend
  path = [list(x)]
  while x != closestart:
    x = prev[x]
    path.append(list(x))
  print("PATH CREATED")
  print("PATH: ",path[::-1])
  return jsonify({"Path":path[::-1]})

@app.route("/test")
def test():
  return "API WORKS."

#def path_finder(slat: float, slong: float, elat: float, elong: float):

#@app.route("/route/<string:slong>/<string:slat>/<string:elong>/<string:elat>", methods = ["GET"])
# def path_finder(slong: str, slat: str, elong: str, elat: str):
#   slat = float(slat)
#   slong = float(slong)
#   elat = float(elat)
#   elong = float(elong)
#   latstart = slat
#   longstart = slong
#   latend = elat
#   longend = elong
#   print("SLAT: ", slat, ", SLONG: ", slong, ", ELAT: ", elat, ", ELONG: ", elong)
#   query = "MATCH (n:LOCATION) RETURN n ORDER BY (n.x - "+str(latstart)+") * (n.x - "+str(latstart)+")" + " + (n.y - " + str(longstart)+")*(n.y - "+str(longstart)+")  ASC LIMIT 1"
#   closestart = (session.run(query))
#   for r in closestart:
#     print(r)
#     start_dict = dict(r["n"])
#     print(start_dict)
#   closestart = (start_dict["x"], start_dict["y"])
#   query = "MATCH (n:LOCATION) RETURN n ORDER BY (n.x - "+str(latend)+") * (n.x - "+str(latend)+")" + " + (n.y - " + str(longend)+")*(n.y - "+str(longend)+")  ASC LIMIT 1"
#   closeend = (session.run(query))
#   for r in closeend:
#     end_dict = dict(r["n"])
#   closeend = (end_dict["x"], end_dict["y"])
#   print("CLOSESTART: ", closestart, " , CLOSEEND: ", closeend)
#   return route(nearest_node(closestart), nearest_node(closeend))

@app.route("/route/<string:slong>/<string:slat>/<string:elong>/<string:elat>", methods = ["GET"])
def path_finder_simple(slong: str, slat: str, elong: str, elat: str):
  start = [float(slong),float(slat)]
  end = [float(elong), float(elat)]
  start = nearest_node(start)
  end = nearest_node(end)
  slong = str(start[0])
  slat = str(start[1])
  elong = str(end[0])
  elat = str(end[1])
  max_nodes=15
  while True:
    seconds = time.time()
    print(max_nodes)
    query = "MATCH paths = (a:LOCATION {x:"+str(slong)+"})-[:ROAD*1.."+str(max_nodes)+"]-(b:LOCATION {x:"+str(elong)+"}) WITH paths, relationships(paths) AS rels UNWIND rels AS rel WITH [metro IN nodes(paths) | metro.x] AS long, [metro IN nodes(paths) | metro.y] AS lat, [metro IN nodes(paths) | metro.INTERSECTION] AS intersection, sum(rel.safety_score) AS travelTime RETURN long, lat, intersection ORDER BY travelTime Limit 1"
    result = (session.run(query))  
    path = list()
    final_path = list()
    for result in result:
      for long in result[0]:
        path.append([long])
      for i, lat in enumerate(result[1]):
        path[i].append(lat)
      for i, intersection in enumerate(result[2]):
        if intersection == True:
          final_path.append(path[i])
    delta_time = time.time()-seconds
    print(delta_time)
    print(final_path)
    if delta_time>2 and final_path!=[]:
      print('done')
      return jsonify(final_path)
    if final_path==[]:
      max_nodes += 3
    else:
      max_nodes+=2

if __name__=="__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)


yo
Nearest Node:  (-122.037200309277, 36.974015258702295)
yo
Nearest Node:  (-122.03805012501235, 36.9739597045977)
15
1.3282139301300049
[]
18
13.728631973266602
[]
21


Failed to read from defunct connection IPv4Address(('neo4j-core-347bdfd6-12.production-orch-0002.neo4j.io', 7687)) (IPv4Address(('104.197.20.211', 7687)))
[2022-03-29 17:09:27,375] ERROR in app: Exception on /route/-122.0368869/36.97403020000001/-122.0380858/36.9741601 [GET]
Traceback (most recent call last):
  File "C:\Users\co41c\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\neo4j\io\_common.py", line 55, in _yield_messages
    buffer.receive(sock, 2)
  File "C:\Users\co41c\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\neo4j\packstream.py", line 482, in receive
    n = sock.recv_into(view[self.used:end], end - self.used)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3312.0_x64__qbz5n2kfra8p0\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Program Files\WindowsApps\Pyt